In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
X = np.load("../COPD_dataset/k-fold-validation/X2_data-002.npy")
y = X[:, -1]
y = y.astype(np.int64)
X = X[:,:-1]
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

X = X.reshape(X.shape[0], X.shape[1], 1)
y = y.reshape(y.shape[0], 1)

In [3]:
def get_model_performance(X, y, model_func, k_splits = 5,epochs=100):
    kfold = KFold(n_splits= k_splits, shuffle=False)
    full_history =  []
    confusion_matrices= []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = model_func(X_train)
        history = model.fit(X_train, y_train, epochs= epochs, validation_data = (X_test, y_test))
        full_history.append(history)
        pred = model.predict(X_test)
        orignal = y_test
        pred = np.argmax(pred, axis=1)
        confusion_matrix = metrics.confusion_matrix(pred, orignal)
        confusion_matrices.append(confusion_matrix)

    return full_history, confusion_matrices

In [4]:
def CNN_Model(data):
    model = keras.Sequential(name="CNN_Sequential")
    model.add(layers.Input(shape = data.shape[1:]))
    model.add(layers.Conv1D(128, 3, activation='relu', name='conv_1'))
    model.add( layers.Conv1D(64, 4, activation="softmax", name="conv_2") )
    model.add(layers.Flatten())
    model.add( layers.Dense(64, activation="relu", name="dense_1") )
    model.add( layers.Dense(128, activation="relu", name="dense_2") )
    model.add( layers.Dense(5, activation="softmax", name="output") )
    model.compile(optimizer=tf.keras.optimizers.SGD(), loss="categorical_crossentropy", metrics=['accuracy','AUC','F1Score','Recall'])
    print(model.summary())
    return model

In [5]:
cnn_history, cnn_confusion_matrices = get_model_performance(model_func= CNN_Model, X= X, y= y, epochs=100)

Model: "CNN_Sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv1D)                 │ (None, 71998, 128)     │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv1D)                 │ (None, 71995, 64)      │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4607680)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │   294,891,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 294,933,893 (1.10 GB)

 Trainable params: 294,933,893 (1.10 GB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node UnsortedSegmentSum defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\ProgramData\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\ProgramData\anaconda3\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\ProgramData\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\ProgramData\an